In [14]:

import pandas as pd
import numpy as np
import polars as pl
import time
import matplotlib.pyplot as plt
from extraction_functions import *

path = '../../../data'


# Load the data
df = pd.read_csv(f'{path}/sensors_data.csv')

print(len(df))

197


### Once file has been opened & imported we will extract the standard WS features first.
### To compartmentalise the code, each cell focuses on one or multiple features within a domain rather than extracting all features from each participant 
### The code starts with step count features first:

In [16]:
#calculate the average number of daily steps by calculating total number of steps divided by the number of days in the analysis
from datetime import datetime
from extraction_functions import *


#add an empty column to the dataframe to add values to it
df['steps'] = np.nan 
df['MVPA steps'] = np.nan
df['Q1'] = 0
df['Q2'] = 0
df['Q3'] = 0
df['Q95'] = 0   


# Loop over each patient file
for index, row in df.iterrows():
    patient_id = row['Patient ID']
    print(f'Processing patient {patient_id}...')

    # Extract numeric patient ID by stripping the 'R' prefix and removing leading zeros
    numeric_patient_id = str(int(patient_id.lstrip('R')))

    # Read the patient data
    patient_file = f'{path}/steps/counts/{numeric_patient_id}_gait_hourly.csv'
    
    try:
        df_patient = pd.read_csv(patient_file)
    except FileNotFoundError:
        print(f"File not found for patient {patient_id}. Skipping.")
        df.at[index, 'steps'] = np.nan
        continue  # Skip to the next iteration if file is missing

    #get the value from the column 'good_wear_hrs' for this patient
    good_wear_hrs = row['Hours of data collected']

    #count the total number of steps in the steps column
    total_steps = df_patient['steps'].sum()
    days = good_wear_hrs/24
    

    #calculate the average number of steps per day
    avg_steps_per_day = total_steps/days
    df.at[index, 'steps'] = avg_steps_per_day


    # Count rows with >= 100 steps to estimate MVPA steps
    mvpa_rows = df_patient[df_patient['steps'] >= 100].shape[0]
    avg_mvpa_per_day = mvpa_rows / days if mvpa_rows > 0 else 0
    df.at[index, 'MVPA steps'] = avg_mvpa_per_day

    #now calculate the Speed-HR ratio

    #find the row with the first value in the column 'walking_time' and row with last value in that column
    first_row = df_patient[df_patient['walking_time'].notnull()].index[0]
    last_row = df_patient[df_patient['walking_time'].notnull()].index[-1]
    df_patient = df_patient.iloc[first_row:last_row+1]

    #open hr data for patient 
    hr_file = f'{path}/hr_values/{patient_id}.npy'
    hr_data = np.load(hr_file)

    hr_resampled = resample_hr_data(hr_data)

    # Create DataFrame for resampled HR data
    df_hr_resampled = pd.DataFrame({'hr': hr_resampled})

    # Ensure length of hr data matches patient data length by taking only first values in hr that match the steps 
    df_hr_resampled = df_hr_resampled.iloc[:len(df_patient)].reset_index(drop=True)


    # Combine the two dataframes
    df_combined = pd.concat([df_patient.reset_index(drop=True), df_hr_resampled], axis=1)

    # Calculate the ratio of steps to HR in df_combined
    df_combined['ratio'] = df_combined['steps'] / df_combined['hr']

    # Form a new DataFrame with only rows where steps > 0 and hr > 0
    df_active = df_combined[(df_combined['steps'] > 0) & (df_combined['hr'] > 0)]

    # Calculate the distribution of CHR ratios for active periods (steps > 50 and hr > 0)
    Q1 = df_active['ratio'].quantile(0.25)
    df.loc[index, 'Q1'] = Q1

    Q2 = df_active['ratio'].median()  # 50th percentile
    df.loc[index, 'Q2'] = Q2

    Q3 = df_active['ratio'].quantile(0.75)
    df.loc[index, 'Q3'] = Q3

    Q95 = df_active['ratio'].quantile(0.95)
    df.loc[index, 'Q95'] = Q95  





Processing patient R001...
Processing patient R002...
Processing patient R003...
Processing patient R004...
Processing patient R005...
Processing patient R006...
Processing patient R007...
Processing patient R008...
Processing patient R009...


/var/folders/m9/nskhz35n09b2_r2sw4dg_kxm0000gn/T/ipykernel_37784/2220637406.py:82: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.106811799607384' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'Q1'] = Q1
/var/folders/m9/nskhz35n09b2_r2sw4dg_kxm0000gn/T/ipykernel_37784/2220637406.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.25862068965517243' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'Q2'] = Q2
/var/folders/m9/nskhz35n09b2_r2sw4dg_kxm0000gn/T/ipykernel_37784/2220637406.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7523084322562723' has dtype incompatible with int64, please explicitly cast to a compatible dt

Processing patient R010...
Processing patient R011...
Processing patient R012...
Processing patient R013...
Processing patient R014...
Processing patient R015...
Processing patient R016...
Processing patient R017...
Processing patient R018...
Processing patient R019...
Processing patient R020...
Processing patient R021...
Processing patient R022...
Processing patient R023...
Processing patient R024...
Processing patient R025...
Processing patient R026...
Processing patient R027...
Processing patient R028...
Processing patient R029...
Processing patient R030...
Processing patient R031...
Processing patient R032...
Processing patient R033...
Processing patient R034...
Processing patient R035...
Processing patient R036...
Processing patient R037...
Processing patient R038...
Processing patient R039...
Processing patient R040...
Processing patient R041...
Processing patient R042...
Processing patient R043...
Processing patient R044...
Processing patient R045...
Processing patient R046...
P

### After step features have been extracted - now movement features will be extracted.

In [18]:
#add several emtpy new columns to the df to add values to
df['Resting HR'] = np.nan
df['Max HR'] = np.nan
df['Min HR'] = np.nan
df['Time in MVPA'] = np.nan
df['Time in LPA'] = np.nan
df['Time in SB'] = np.nan
df['MVPA HR'] = np.nan
df['LPA HR'] = np.nan
df['SB HR'] = np.nan


# Load the data
print('starting_loop')
for index, row in df.iterrows():
    patient_id = row['Patient ID']
    file_name = df[df['Patient ID'] == patient_id]['file_name'].values[0]
    start_time = df[df['Patient ID'] == patient_id]['Start'].values[0]
    print(f'Starting patient {patient_id}')

    #get the good qual wear time from the column good_wear_hrs and find this in days 
    good_wear_hrs = df[df['Patient ID'] == patient_id]['Hours of data collected'].values[0]
    days = good_wear_hrs / 24
    print(f'Days: {days}')

    # Load the HR data
    file = f'{path}/hr_values/{patient_id}.npy'
    hr_values = np.load(file)

    #open the activity classification file
    file = f'{path}/activity_class/{patient_id}_combined-timeSeries.csv.gz'
    acc_df = pd.read_csv(file, compression='gzip')

    #upsample to hr data to match the length of the activity classification data
    hr_30s_values = average_hr_30s(hr_values)

    # Align HR and accelerometer data
    acc_df = align_hr_and_acc(hr_30s_values, acc_df)

    # Extract sleep data
    sleep_df = extract_sleep_data(acc_df)

    # Filter the rows where the HR column is not 0
    sleep_df = sleep_df[sleep_df['HR'] != 0]
    if len(sleep_df) > 0:
        # Calculate the average and add to column
        resting_hr = sleep_df['HR'].mean()
        print(f"Resting HR during sleep:{resting_hr}")
        df.at[index, 'Resting HR'] = resting_hr
    else:
        print('No resting HR found')
        #calculate the resting HR from the average period when all rows have a 1 in the sedentary column
        resting_hr = acc_df[acc_df['sedentary'] == 1]['HR'].mean()
        #print(f"Resting HR from sedentary period:{resting_hr}")
        df.at[index, 'Resting HR'] = resting_hr

    
    #get the maximum value of the HR column
    max_hr = acc_df['HR'].max()
    df.at[index, 'Max HR'] = max_hr

    #get the minimum value of the HR column that is not 0
    min_hr = acc_df[acc_df['HR'] != 0]['HR'].min()
    df.at[index, 'Min HR'] = min_hr

    #get the time in minutes spent in MVPA, LPA, VPA and SB (each row is 30s) along with the average HR for each
    time_in_mvpa = (acc_df[acc_df['moderate-vigorous'] == 1].shape[0] / 2) / days
    print(f'Time in MVPA: {time_in_mvpa}')
    df.at[index, 'Time in MVPA'] = time_in_mvpa
    mvpa_hr = acc_df[(acc_df['moderate-vigorous'] == 1) & (acc_df['HR'] != 0)]['HR'].mean()
    df.at[index, 'MVPA HR'] = mvpa_hr

    time_in_lpa = (acc_df[acc_df['light'] == 1].shape[0] / 2) / days
    df.at[index, 'Time in LPA'] = time_in_lpa
    lpa_hr = acc_df[(acc_df['light'] == 1) & (acc_df['HR'] != 0)]['HR'].mean()
    df.at[index, 'LPA HR'] = lpa_hr

    time_in_sb = (acc_df[acc_df['sedentary'] == 1].shape[0] / 2) / days
    df.at[index, 'Time in SB'] = time_in_sb
    sb_hr = acc_df[(acc_df['sedentary'] == 1) & (acc_df['HR'] != 0)]['HR'].mean()
    df.at[index, 'SB HR'] = sb_hr


starting_loop
Starting patient R001
Days: 3.0529879629629626
Trimmed 1 values
Resting HR during sleep:64.72551928783383
Time in MVPA: 76.97377220312706
Starting patient R002
Days: 2.353447685185185


/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])


Trimmed 1 values
Resting HR during sleep:72.11278586278586
Time in MVPA: 5.523810910195385
Starting patient R003
Days: 3.429448611111111
Trimmed 1 values
Resting HR during sleep:101.82392131979695
Time in MVPA: 21.577812759825743
Starting patient R004
Days: 2.1227560185185186
Trimmed 1 values


/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a fo

Resting HR during sleep:76.54208542713567
Time in MVPA: 95.63039663016697
Starting patient R005
Days: 1.7165472222222224
Trimmed 1 values
Resting HR during sleep:73.64935064935065
Time in MVPA: 15.729249769805987
Starting patient R006
Days: 2.998518518518518
Trimmed 1 values
Resting HR during sleep:75.63511611541168
Time in MVPA: 2.834733201581028
Starting patient R007
Days: 3.010323148148148


...

/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a fo

Trimmed 1 values
Resting HR during sleep:68.38555030384875
Time in MVPA: 26.482367085129418
Starting patient R193
Days: 3.081481481481481
Trimmed 1 values
Resting HR during sleep:62.545505617977526
Time in MVPA: 2.1093750000000004
Starting patient R194
Days: 3.045043981481482
Resting HR during sleep:78.63376932223544
Time in MVPA: 32.01924195280886
Starting patient R195
Days: 3.0103703703703704
Trimmed 1 values


/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a fo

Resting HR during sleep:68.99583622484386
Time in MVPA: 8.470718503937007
Starting patient R196
Days: 3.057777777777778
Trimmed 1 values
Resting HR during sleep:64.99224259520452
Time in MVPA: 0.0
Starting patient R197
Days: 3.034074074074074
Resting HR during sleep:67.12127512127512
Time in MVPA: 39.88037109375
Starting patient R198
Days: 2.986666666666667
Trimmed 1 values
Resting HR during sleep:74.25898617511521
Time in MVPA: 26.953125


/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  acc_df['time_cleaned'] = pd.to_datetime(acc_df['time'].str.split("[").str[0])
/Users/aronsyversen/Desktop/REMOTES/CPET-pred/src/feature_extraction/extraction_functions.py:235: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a fo

In [13]:
df.head()

,Patient ID,Biosensor ID,file_name,Start,Wear time,missing ACC (% of total signal),missing ECG overall (% of total signal),Missing ECG from electrode (% of total signal),ecg_qual,wear_hrs,...,Q95,Resting HR,Max HR,Min HR,Time in MVPA,Time in LPA,Time in SB,MVPA HR,LPA HR,SB HR
0,R001,AVNDI,01_10,2022-12-21 14:00:00,74.240000,1.304268,9.743286,8.439017,75.514480,74.240000,...,1.048475,64.725519,159.0,57.0,76.973772,193.908396,665.413695,93.297357,87.536122,78.532001
1,R002,AVNDM,01_10,2023-01-03 12:15:00,67.982222,16.915419,17.204830,0.289411,74.175622,67.982222,...,0.633188,72.112786,146.0,54.0,5.523811,35.054954,1028.916009,110.694444,101.817797,86.742545
2,R003,AVNDG,01_10,2023-01-09 10:32:00,94.720000,13.105187,13.116111,0.010923,85.782574,94.720000,...,0.492181,101.823921,163.0,72.0,21.577813,50.153835,1004.680458,115.473684,116.899068,105.081097
3,R004,BBJMP,01_10,2023-01-17 12:50:00,79.928889,36.260662,36.298541,0.037879,60.794467,79.928889,...,0.942646,76.542085,157.0,67.0,95.630397,95.865939,1197.970929,122.030343,104.098462,93.150528
4,R005,BGFJQ,01_10,2023-01-23 16:04:00,47.786667,13.789481,14.579876,0.790395,54.630006,47.786667,...,0.482796,73.649351,149.5,62.0,15.729250,102.822688,889.285177,76.000000,87.709030,82.015659


In [19]:
# save the dataframe to a csv file

df.to_csv(f'{path}/sensors_data.csv', index=False)